### Cu Estimator example

This notebook demonstrates work of the Copper Estimator.
Current version (v1) of the estimator based on the numerical solution of numerical solution of a system of equations. `least_squares` is used as a loss function. Please check [the docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html#scipy-optimize-least-squares) for more details.

**Important:**
Don't forget to install our application as a pip package. Use `pip install -e .` in the project root directory.

In [4]:
import json
from src.cu_estimator import CuEstimator
from pandas.io.json import json_normalize

In [5]:
with open('../data/1/previous_heats_with_properties.json') as f:
    data = json.load(f)
df = json_normalize(data)
df = df.rename(columns=lambda col: col.rpartition('.')[-1])
df.head()

,heat_seq,heat_id,steel_grade,required_weight,tap_weight,cu_pct,bushling,pig_iron,municipal_shred,skulls
0,51,heat-501,ST1,1000,970,0.095,300,200,350,200
1,51,heat-502,ST1,1000,920,0.104,300,200,300,250
2,51,heat-503,ST1,1000,940,0.105,300,200,300,250
3,51,heat-504,ST1,1000,935,0.089,300,200,350,200
4,51,heat-505,ST1,1000,945,0.093,300,200,350,200


---

We need to train the estimator on the dataset.

In [6]:
estimator = CuEstimator()
estimator.fit(df)

Now we can predict the amount of copper for the given recipe.

_In fact, you should never use the same data for train and evaluation=)_

In [9]:
estimator.predict(df.iloc[2])

0.10449999984942215